In [12]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784')
mnist.data.shape, mnist.target.shape

((70000, 784), (70000,))

In [13]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt

In [14]:
X,y= mnist['data'], mnist['target']

* 데이터가 커서 일부만 사용

In [15]:
X_train, X_test, y_train, y_test = X[:10000],X[10000:15000],y[:10000],y[10000:15000]

In [16]:
np.random.seed(42)
rnd_idx= np.random.permutation(10000)
X_train=X_train[rnd_idx]
y_train=y_train[rnd_idx]


## StandardScaler

In [17]:
scaler=StandardScaler()
X_train_scaled= scaler.fit_transform(X_train.astype(np.float32))
X_test_scaled= scaler.fit_transform(X_test.astype(np.float32))

lin_clf=LinearSVC(max_iter=30000, random_state=42)
lin_clf.fit(X_train_scaled,y_train)

C:\Users\seoyounglee\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


LinearSVC(max_iter=30000, random_state=42)

In [18]:
y_pred=lin_clf.predict(X_train_scaled)
accuracy_score(y_train, y_pred)

0.978

## SVC

In [19]:
svm_clf= SVC(gamma='auto')
svm_clf.fit(X_train_scaled[:3000],y_train[:3000])

SVC(gamma='auto')

In [20]:
y_pred=svm_clf.predict(X_train_scaled)
accuracy_score(y_train, y_pred)

0.9349

## Hyperparameter
* 일단 데이터셋을 적게 설정해서 결과를 보았다
* 위에서 스케일한 데이터 이용
* N=1000

In [30]:
gamma1, gamma2 = 0.01, 0.005
C1, C2,C3 =1, 5, 10
hyperparams = (gamma1, C1), (gamma1, C2),(gamma1, C3),  (gamma2, C1), (gamma2, C2) ,(gamma2, C3)

svm_clfs = []
for gamma, C in hyperparams:
    rbf_kernel_svm_clf = Pipeline([
            
            ("svm_clf", SVC(kernel="rbf", gamma=gamma, C=C))
        ])
    rbf_kernel_svm_clf.fit(X_train_scaled[:1000],y_train[:1000])
    y_pred=rbf_kernel_svm_clf.predict(X_train_scaled[:1000])
    print('accuracy_score: ',accuracy_score(y_train[:1000], y_pred))
    #svm_clfs.append(rbf_kernel_svm_clf)

accuracy_score:  0.961
accuracy_score:  1.0
accuracy_score:  1.0
accuracy_score:  0.906
accuracy_score:  0.99
accuracy_score:  1.0


* N=5000 감마 0.01 C=10 일 때 99.96%

In [31]:
gamma1, gamma2 = 0.01, 0.005
C1, C2,C3 =1, 5, 10
hyperparams = (gamma1, C1), (gamma1, C2),(gamma1, C3),  (gamma2, C1), (gamma2, C2) ,(gamma2, C3)

svm_clfs = []
for gamma, C in hyperparams:
    rbf_kernel_svm_clf = Pipeline([
            
            ("svm_clf", SVC(kernel="rbf", gamma=gamma, C=C))
        ])
    rbf_kernel_svm_clf.fit(X_train_scaled[:5000],y_train[:5000])
    y_pred=rbf_kernel_svm_clf.predict(X_train_scaled[:5000])
    print('accuracy_score: ',accuracy_score(y_train[:5000], y_pred))

accuracy_score:  0.939
accuracy_score:  0.9936
accuracy_score:  0.9996
accuracy_score:  0.8282
accuracy_score:  0.9684
accuracy_score:  0.9888


* N=10000 감마 0.01 C=10 일 때 99.86%

In [36]:
gamma1, gamma2 = 0.01, 0.005
C1, C2,C3 =1, 5, 10
hyperparams = (gamma1, C1), (gamma1, C2),(gamma1, C3),  (gamma2, C1), (gamma2, C2) ,(gamma2, C3)

svm_clfs = []
for gamma, C in hyperparams:
    rbf_kernel_svm_clf = Pipeline([
            
            ("svm_clf", SVC(kernel="rbf", gamma=gamma, C=C))
        ])
    rbf_kernel_svm_clf.fit(X_train_scaled[:10000],y_train[:10000])
    y_pred=rbf_kernel_svm_clf.predict(X_train_scaled[:10000])
    print('accuracy_score: ',accuracy_score(y_train[:10000], y_pred))

accuracy_score:  0.9275
accuracy_score:  0.9876
accuracy_score:  0.9986
accuracy_score:  0.7917
accuracy_score:  0.9601
accuracy_score:  0.9792


## >> 위의 결과를 통해 감마=0.01 C=10 일 때 가장 좋은 결과를 보임을 확인

In [ ]:
#시각화

In [ ]:
plt.figure(figsize=(11, 7))

for i, svm_clf in enumerate(svm_clfs):
    plt.subplot(221 + i)
    plot_predictions(rbf_kernel_svm_clf, [-1.5, 2.5, -1, 1.5])
    plot_dataset(X, y, [-1.5, 2.5, -1, 1.5])
    gamma, C = hyperparams[i]
    plt.title(r"$\gamma = {}, C = {}$".format(gamma, C), fontsize=16)

plt.show()

In [ ]:
'''
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform

param= {'gamma': reciprocal(0.001,0.005), 'C':uniform(1, 5)}
rnd_search_cv=  RandomizedSearchCV(svm_clf, param, cv=3, n_iter=10, n_jobs=-1)
rnd_search_cv.fit(X_trian_scaled[:1000],y_train[:1000])
rnd_serach_cv.best_estimator_
rnd_search_cv.best_score_
rnd_search_cv.fit(X_trian_scaled,y_train)
y_pred=rnd_search_cv.best_estimator_.predict(X_test_scaled)
accuracy_score(y_test, y_pred)
''